In [3]:
import os
# from dotenv import load_dotenv


from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
# from langchain_community.llms import Ollama
# from langchain_community.embeddings import OllamaEmbeddings

# MODEL="gpt-3.5-turbo"
# MODEL="llama2"
# model = Ollama(model=MODEL)
# embeddings = OllamaEmbeddings()
MODEL="gpt-4"
model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model=MODEL)
embeddings = OpenAIEmbeddings()


In [4]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('agent-dojo.pdf')
pages = loader.load_and_split()
pages

In [6]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()

retriever.invoke("Agents")

from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model 
    | parser
)

chain.invoke({"question": "what is this paper about?, Give me a small description"})

In [ ]:
questions = [
    "What is the crux of this paper?",
    "What tool is being discussed in the paper?",
    "What is the conclusion about security against prompt injection attacks?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase
from deepeval import evaluate

# Initialize the metric
answerRelevancy = AnswerRelevancyMetric()
# faithfulness = FaithfulnessMetric()


question = "What does Agent Dojo do?"
retrieved_docs = retriever.get_relevant_documents(question)
retrieved_texts = [doc.page_content for doc in retrieved_docs]
generated_answer = chain.invoke({'question': question})

test_case = LLMTestCase(
    input=question,
    actual_output=generated_answer,   # this is the LLM output
    retrieval_context=retrieved_texts
)

evaluate([test_case], [AnswerRelevancyMetric()])
